## 8.1.1 Loading image classification dataset


```
train/
  0/
    1.png
    21.png
    ...
  1/
  2/
  3/
  ...

test/
  0/
  1/
  ...
```


In [1]:
import os
import autokeras as ak

batch_size = 32
img_height = 28
img_width = 28

parent_dir = "data"

test_data = ak.image_dataset_from_directory(
    os.path.join(parent_dir, "test"),
    seed=123,
    color_mode="grayscale",
    image_size=(img_height, img_width),
    batch_size=batch_size,
)
for images, labels in test_data.take(1):
    print(images.shape, images.dtype)
    print(labels.shape, labels.dtype)


Using TensorFlow backend
Found 10000 files belonging to 10 classes.
(32, 28, 28, 1) <dtype: 'float32'>
(32,) <dtype: 'string'>


## 8.1.2 Splitting the loaded dataset


In [2]:
all_train_data = ak.image_dataset_from_directory(
    os.path.join(parent_dir, "train"),
    seed=123,
    color_mode="grayscale",
    image_size=(img_height, img_width),
    batch_size=batch_size,
)
train_data = all_train_data.take(int(60000 / batch_size * 0.8))
validation_data = all_train_data.skip(int(60000 / batch_size * 0.8))


Found 60000 files belonging to 10 classes.


In [3]:
train_data = ak.image_dataset_from_directory(
    os.path.join(parent_dir, "train"),
    validation_split=0.2,
    subset="training",
    seed=123,
    color_mode="grayscale",
    image_size=(img_height, img_width),
    batch_size=batch_size,
)

validation_data = ak.image_dataset_from_directory(
    os.path.join(parent_dir, "train"),
    validation_split=0.2,
    subset="validation",
    seed=123,
    color_mode="grayscale",
    image_size=(img_height, img_width),
    batch_size=batch_size,
)


Found 60000 files belonging to 10 classes.
Using 48000 files for training.
Found 60000 files belonging to 10 classes.
Using 12000 files for validation.


In [4]:
import tensorflow as tf

train_data = train_data.prefetch(5)
validation_data = validation_data.prefetch(5)
test_data = test_data.prefetch(tf.data.AUTOTUNE)


Then we just do one quick demo of AutoKeras to make sure the dataset works.


In [5]:
clf = ak.ImageClassifier(overwrite=True, max_trials=1)
clf.fit(train_data, epochs=1, validation_data=validation_data)
print(clf.evaluate(test_data))


Trial 1 Complete [00h 03m 36s]
val_loss: 0.06069127842783928

Best val_loss So Far: 0.06069127842783928
Total elapsed time: 00h 03m 36s
1500/1500 [==============================] - 193s 128ms/step - loss: 0.1766 - accuracy: 0.9456 - val_loss: 0.0642 - val_accuracy: 0.9813


313/313 [==============================] - 12s 38ms/step - loss: 0.0538 - accuracy: 0.9829
[0.0537872239947319, 0.9829000234603882]


## 8.1.3 Loading text classification dataset
You can also load text datasets in the same way.


In [6]:
!!wget https://github.com/datamllab/automl-in-action-notebooks/raw/master/data/imdb.tar.gz
!!tar xzf imdb.tar.gz


[]

For this dataset, the data is already split into train and test.
We just load them separately.


In [7]:
import os
import autokeras as ak
import tensorflow as tf

train_data = ak.text_dataset_from_directory(
    "imdb/train",
    validation_split=0.2,
    subset="training",
    seed=123,
    max_length=1000,
    batch_size=32,
).prefetch(1000)

validation_data = ak.text_dataset_from_directory(
    "imdb/train",
    validation_split=0.2,
    subset="validation",
    seed=123,
    max_length=1000,
    batch_size=32,
).prefetch(1000)

test_data = ak.text_dataset_from_directory(
    "imdb/test",
    max_length=1000,
).prefetch(1000)


Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.


In [8]:
clf = ak.TextClassifier(overwrite=True, max_trials=1)
clf.fit(train_data, epochs=2, validation_data=validation_data)
print(clf.evaluate(test_data))


Trial 1 Complete [00h 06m 46s]
val_loss: 0.34085893630981445

Best val_loss So Far: 0.34085893630981445
Total elapsed time: 00h 06m 46s
Epoch 1/2
625/625 [==============================] - 190s 302ms/step - loss: 0.4992 - accuracy: 0.7318 - val_loss: 0.3478 - val_accuracy: 0.8450
Epoch 2/2
625/625 [==============================] - 190s 303ms/step - loss: 0.2980 - accuracy: 0.8743 - val_loss: 0.3488 - val_accuracy: 0.8510


782/782 [==============================] - 65s 83ms/step - loss: 0.3442 - accuracy: 0.8537
[0.3442336320877075, 0.8537200093269348]


## 8.1.4 Handling large dataset in general format


In [9]:
data = [5, 8, 9, 3, 6]


def generator():
    for i in data:
        yield i


for x in generator():
    print(x)


5
8
9
3
6


In [10]:
dataset = tf.data.Dataset.from_generator(generator, output_types=tf.int32)
for x in dataset:
    print(x.numpy())


5
8
9
3
6


In [11]:
import numpy as np

parent_dir = "imdb"


def load_data(path):
    data = []
    for class_label in ["pos", "neg"]:
        for file_name in os.listdir(os.path.join(path, class_label)):
            data.append((os.path.join(path, class_label, file_name), class_label))

    data = np.array(data)
    np.random.shuffle(data)
    return data


def get_generator(data):
    def data_generator():
        for file_path, class_label in data:
            text_file = open(file_path, "r")
            text = text_file.read()
            text_file.close()
            yield text, class_label

    return data_generator


all_train_np = load_data(os.path.join(parent_dir, "train"))


def np_to_dataset(data_np):
    return (
        tf.data.Dataset.from_generator(
            get_generator(data_np),
            output_types=tf.string,
            output_shapes=tf.TensorShape([2]),
        )
        .map(lambda x: (x[0], x[1]))
        .batch(32)
        .prefetch(5)
    )


train_data = np_to_dataset(all_train_np[:20000])
validation_data = np_to_dataset(all_train_np[20000:])
test_np = load_data(os.path.join(parent_dir, "test"))
test_data = np_to_dataset(test_np)

for texts, labels in train_data.take(1):
    print(texts.shape)
    print(labels.shape)


(32,)
(32,)


In [12]:
clf = ak.TextClassifier(overwrite=True, max_trials=1)
clf.fit(train_data, epochs=2, validation_data=validation_data)
print(clf.evaluate(test_data))


Trial 1 Complete [00h 09m 11s]
val_loss: 0.2792743146419525

Best val_loss So Far: 0.2792743146419525
Total elapsed time: 00h 09m 11s
Epoch 1/2
625/625 [==============================] - 218s 346ms/step - loss: 0.4668 - accuracy: 0.7531 - val_loss: 0.3075 - val_accuracy: 0.8704
Epoch 2/2
625/625 [==============================] - 208s 333ms/step - loss: 0.2508 - accuracy: 0.8985 - val_loss: 0.2710 - val_accuracy: 0.8892


782/782 [==============================] - 79s 100ms/step - loss: 0.2785 - accuracy: 0.8890
[0.27845025062561035, 0.8889999985694885]
